In [1]:
!git clone https://github.com/bcgsc/AMPlify.git
import tensorflow as tf
import numpy as np
import pandas as pd
pip install Bio
!cp -r AMPlify/src/* `pwd`
import AMPlify as ActivityPredictor

Cloning into 'AMPlify'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 247 (delta 23), reused 46 (delta 14), pack-reused 187
Receiving objects: 100% (247/247), 274.23 MiB | 20.80 MiB/s, done.
Resolving deltas: 100% (103/103), done.
Updating files: 100% (31/31), done.


In [7]:
anti_model = ActivityPredictor.build_amplify()
anti_model.load_weights("AMPlify/models/balanced/AMPlify_balanced_model_weights_1.h5", by_name=True)

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [49]:
class GRUDecoder(tf.keras.Model):
    def __init__(self, hidden_dim):
        super().__init__()
        self.RepeatVector = tf.keras.layers.RepeatVector(200, name="RepeatVector")
        self.GRU = tf.keras.layers.GRU(hidden_dim, name="GRU_0", return_sequences=True)
        self.dense = tf.keras.layers.Dense(20, activation="softmax", name="Output")

    def call(self, inputs):
        x = self.RepeatVector(inputs)
        x = self.GRU(x)
        x = self.dense(x)
        return x

class Agent(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.decoder = GRUDecoder(hidden_dim=10)
        self.encoder = tf.keras.layers.Dense(20, activation="relu", name="encoder")

    def call(self, state):
        x = self.encoder(state)
        return self.decoder(x)

def compute_reward(generated_sequence):
    pred = anti_model(generated_sequence)
    return pred

def train_step(agent, optimizer, state):
    with tf.GradientTape() as tape:
        generated_sequence = agent(state)
        reward = compute_reward(generated_sequence)
        loss = 1-reward  # We want to maximize the reward

    gradients = tape.gradient(loss, agent.trainable_variables)
    optimizer.apply_gradients(zip(gradients, agent.trainable_variables))

agent = Agent()
optimizer = tf.keras.optimizers.Adam()

In [78]:
# Assume some initial state for the dialogue
initial_state = np.array([[1,0,0,0,0]])

# Train the agent
for epoch in range(10):  # Assume 10 training epochs
    train_step(agent, optimizer, initial_state)

In [52]:
pred = agent(initial_state)

In [77]:
anti_model(pred)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.99883604]], dtype=float32)>

In [70]:
from sklearn.preprocessing import LabelEncoder

In [72]:
aa = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']
labelencoder = LabelEncoder.fit(aa)

TypeError: ignored

In [79]:
tf.math.argmax(pred, axis=2)[0]

<tf.Tensor: shape=(200,), dtype=int64, numpy=
array([18, 18, 18, 18,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7])>

In [74]:
aa = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']

In [76]:
aa[7]

'I'